In [1]:
# Load in libraries

import numpy as np
import pandas as pd
import os
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

# load in directories
from experiment_code.constants import Defaults

## Make spreadsheet

In [2]:
os.chdir(Defaults.STIM_DIR)

# other option is to hardcode name of csv
file_list = glob.glob('*')

# load in peele spreadsheet
df = pd.read_csv(file_list[0])

In [3]:
# add in new sentence columns

df['sentence'] = df['sentence'].str.extract(pat = "([A-Za-z ,']+)")
df['full_sentence'] = df['sentence'] + '' + df['target word']
                                          

In [4]:
df['full_sentence'].iloc[11]

"He called home from college saying he'd met a girl"

In [5]:
# hardcode task parameters

num_sentences_per_block = 180
trial_dur_ms = 10000
iti_dur = 500

# number of trials per block
trials_per_block = np.arange(start=1, stop=len(df) - 1, step=num_sentences_per_block)

In [6]:
# make gorilla spreadsheet

outname = Defaults.TARGET_DIR / f'all_blocks_{num_sentences_per_block}_trials.csv'

df_all = pd.DataFrame()
for i, block in enumerate(trials_per_block[:-1]):
    df_new = []
    df_new = df.iloc[trials_per_block[i]:trials_per_block[i+1]]
    df_new['display'] = 'trial'
    df_new['iti_dur_ms'] = iti_dur
    df_new['trial_dur_ms'] = trial_dur_ms
    df_new['randomise_blocks'] = i+1
    
    # if it's the first block - set instructions
    if i == 0: 
        df_new = pd.concat([pd.DataFrame([{'display': 'instructions'}]), df_new], ignore_index=False, sort=True)
    elif i == len(trials_per_block)-2: 
        df_new = df_new.append([{'display': 'end'}], ignore_index=True, sort=False)
    
    df_all = pd.concat([df_all, df_new], ignore_index=False, sort=False)
    
df_all.to_csv(outname, header=True, index=False)